In [ ]:
import sys
sys.path.append("../")
import anndata
import pandas as pd
import scanpy as sc
import numpy as np
from scipy import stats
from scipy.sparse import issparse
import trvae
from trvae import pl
from trvae.models._trvae import CLDRCVAE

## loading and preparing data 

In [ ]:
adata = sc.read("../data/kang_normalized.h5ad")
n_conditions = adata.obs["condition"].unique().shape[0]

In [ ]:
condition_key = "condition"
cell_type_key = "cell_type"

In [ ]:
adata_train = adata[~((adata.obs["cell_type"] == "CD4 T") & (adata.obs["condition"] == "stimulated"))]

In [ ]:
sc.pl.umap(adata_train, color=["condition", "cell_type"], wspace=.4)

## creating model object 

In [ ]:
conditions = adata.obs[condition_key].unique().tolist()
conditions

cell_types = adata.obs[cell_type_key].unique().tolist()
cell_types

In [ ]:
network = CLDRCVAE(
    gene_size=adata.shape[1],
    architecture=[512, 256, 128],
    n_topic=50,
    gene_names=adata.var_names.tolist(),
    conditions=conditions,
    cell_types=cell_types, 
    cell_type_key=cell_type_key,
    model_path='./models/CLDRCVAE/Kang/',
    dropout_rate=0.1,
    alpha=0.0001,
    beta=60,
    eta=30,
    contrastive_lambda=15.0,
    second_contrastive_lambda=8.0,
    topk=5,
    loss_fn='sse', 
    output_activation='relu'
)

### Training CLDRCVAE

In [ ]:
network.train(adata,
              condition_key,
              train_size=0.8,
              n_epochs=300,
              batch_size=512,
              early_stop_limit=50,
              lr_reducer=20,
              verbose=5,
              save=False
              )

## visualizing the latent space

In [ ]:
latent_y = network.get_latent(
    adata=adata, 
    batch_key="condition",  
    return_z=True
)

adata_latent = sc.AnnData(latent_y)
adata_latent.obs["cell_type"] = adata.obs["cell_type"].tolist()
adata_latent.obs["condition"] = adata.obs["condition"].tolist()

sc.pp.neighbors(adata_latent)

sc.tl.umap(adata_latent)

sc.pl.umap(adata_latent, color=["condition", "cell_type"])

## Making prediction

Evaluate the similarity between the predicted data generated by the model and the real data, and quantify the proximity of the generated data to the real data through correlation.

In [ ]:
def calc_R2(pred_adata, real_adata, n_trials=1000):
    r_values_mean = np.zeros((n_trials,))
    r_values_var = np.zeros((n_trials,))
    
    for i in range(n_trials):
        pred_idx = np.random.choice(range(pred_adata.shape[0]), int(0.9 * pred_adata.shape[0]), replace=False)
        real_idx = np.random.choice(range(real_adata.shape[0]), int(0.9 * real_adata.shape[0]), replace=False)
        
        if issparse(pred_adata.X):
            pred_adata.X = pred_adata.X.A
            real_adata.X = real_adata.X.A

        pred_mean = np.mean(pred_adata.X[pred_idx], axis=0)
        real_mean = np.mean(real_adata.X[real_idx], axis=0)
        pred_var = np.var(pred_adata.X[pred_idx], axis=0)
        real_var = np.var(real_adata.X[real_idx], axis=0)

        _, _, r_value_mean, _, _ = stats.linregress(pred_mean, real_mean)
        _, _, r_value_var, _, _ = stats.linregress(pred_var, real_var)

        r_values_mean[i] = r_value_mean ** 2
        r_values_var[i] = r_value_var ** 2

    return (
        r_values_mean.mean(), r_values_mean.std(),
        r_values_var.mean(), r_values_var.std()
    )

In [ ]:
ground_truth = adata[(adata.obs["cell_type"] == "CD4 T")]

adata_source = adata[(adata.obs["cell_type"] == "CD4 T") &
                     (adata.obs["condition"] == "control")]

predicted_data = network.predict(
    adata = adata_source, 
    condition_key="condition", 
    target_condition="stimulated"
)

In [ ]:
adata_pred = sc.AnnData(predicted_data)
adata_pred.obs["condition"] = np.tile("predicted", len(adata_pred))
adata_pred.var_names = adata_source.var_names.tolist()

all_adata = ground_truth.concatenate(adata_pred)

In [ ]:
sc.tl.pca(all_adata)
sc.pl.pca(all_adata, color=["condition"])

In [ ]:
sc.pl.violin(all_adata, keys="ISG15", groupby="condition")

In [ ]:
r2_mean, r2_mean_std, r2_var, r2_var_std = calc_R2(adata_pred, ground_truth)

print(f"R² Mean: {r2_mean}, R² Mean Std: {r2_mean_std}")
print(f"R² Var: {r2_var}, R² Var Std: {r2_var_std}")

## Mean correlation plot

In [ ]:
cell_type_adata = all_adata[all_adata.obs["cell_type"] == "CD4 T"]

In [ ]:
sc.tl.rank_genes_groups(cell_type_adata, reference='control', 
                       groupby=condition_key, groups=["stimulated"],
                       key_added='up_reg_genes', n_genes=50, method='wilcoxon')
sc.tl.rank_genes_groups(cell_type_adata, reference="stimulated",
                         groupby=condition_key, groups=['control'],
                         key_added='down_reg_genes', n_genes=50, method='wilcoxon')
up_genes = cell_type_adata.uns['up_reg_genes']['names']['stimulated']
down_genes = cell_type_adata.uns['down_reg_genes']['names']['control']
top_genes = up_genes.tolist() + down_genes.tolist()

print(f"Top genes: {top_genes}")
print(f"Number of top genes: {len(top_genes)}")
print(all_adata.obs["condition"].unique())


In [ ]:
trvae.pl.reg_mean_plot(all_adata,
                         top_100_genes=top_genes,
                         gene_list=top_genes[:5] + top_genes[50:55],
                         condition_key=condition_key,
                         axis_keys={'x': 'predicted', 'y': 'stimulated'},  # 更新标签匹配
                         labels={'x': "", 'y': ""},
                         legend=False,
                         show=True,
                         x_coeff=1.0,
                         y_coeff=0.0)

In [ ]:
trvae.pl.reg_var_plot(all_adata,
                         top_100_genes=top_genes,
                         gene_list=top_genes[:5] + top_genes[50:55],
                         condition_key=condition_key,
                         axis_keys={'x': 'predicted', 'y': 'stimulated'},  # 更新标签匹配
                         labels={'x': "", 'y': ""},
                         legend=False,
                         show=True,
                         x_coeff=1.0,
                         y_coeff=0.1)